# Métodos "clásicos" de aprendizaje estadístico

## Este notebook contiene 3 ejercicios:
1. Hacer nuestra propia implementación de KNN-I.
2. Contruir un pipeline para aplicar KNN-I usando scikit-learn.
3. Implementar MissForest usando los bloques de construcción que ofrece scikit-learn.
4. Elegir un conjunto de metricas y comparar las 2 tecnicas !

Primero importemos todas las dependencias necesarias

In [2]:
import time
import copy
import math
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

Luego, importemos los datos meteorológicos

In [3]:
marengo_df = pd.read_csv("src/marengo_nans.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'src/marengo_nans.csv'

## KNN-I

Gran parte de los algoritmos de aprendizaje automático estan implementados en elegantes bibliotecas que permiten su rapida ejecución bajo el tradicional formato de dos métodos: ```.fit()``` y ```.predict()```.Esto reduce los algoritmos a una caja negra que algunos usuarios utilizan indiscriminadamente. La falta de comprensión del funcionamiento de los métodos es un riesgo para la calidad de los resultados.

Aprovechando que KNN es un algoritmo de facil implementación, vamos a escribirlo nosotros mismos para interiorizar el funcionamiento. En un primer momento vamos a hacerlo con python puro.

### Ex.1: Hacer nuestra propia implementación de KNN-I.

Como lo mencionamos durante la presentación el algoritmo de los KNN consta de tres étapas.

#### 1.Cálculo de la distancia entre observaciones.
**Nota:** Nuestra implementación sera secuencial y busca ser lo más simple posible, para hacerla mas comprensible. Por lo tanto sacrificaremos la complejidad computacional de nuestro algoritmo.

In [4]:
def eucledian_distance_nans(a, b):
    """
    Una función que calcule la distancia euclidiana entre dos vectores.
    La distancia euclidiana entre dos puntos en un espacio n-dimensional es la raíz cuadrada de
    la suma de las diferencias al cuadrado entre las coordenadas correspondientes de los puntos.
    Esta implementación debe ser robusta a los valores faltantes dentro de los vectores.
    """
    # Paso 1: Crear una mascara que indique los elementos que son NaN en uno o ambos vectores.
    mascara = []
    for i,j in zip(a, b):
        if i is None or j is None:
            mascara.append(False)
        else:
            mascara.append(True)

    # TODO Paso 2: Estimar el ponderador contando los elementos que son Nan: # elemetos/# elementos comparables.
    weight = len(mascara)/mascara.count(True)

    # Paso 3: Calcular la distancia eucladiana considerando solo los elementos no nulos. Devolver dicha distancia.
    sumatoria = 0
    for idx in range(len(a)):
        # TODO Calcular la sumatoria al cuadrado interna de la distancia eucladiana incluyendo solo los elementos no nulos
        if mascara[idx]:
            sumatoria += (a[idx]-b[idx])**2

    return round(math.sqrt(weight * sumatoria), 8)

In [5]:
vec_1 = [0., 10., 5., None, 5., 1., 5., None]
vec_2 = [1., None, 0., 7., 0., 2., 3., 10.]
vec_3 = [15., 120., 10., 1., 2., 7., 15., 8.]
print(eucledian_distance_nans(vec_1, vec_2))
print(eucledian_distance_nans(vec_2, vec_3))
print(eucledian_distance_nans(vec_3, vec_1))

# LA RESPUESTA DEBE SER:
# 9.46572765
# 24.11875382
# 129.0736224

9.46572765
24.11875382
129.0736224


In [37]:
def calculate_distance_matrix(X):
    ref_len = len(X[0])
    for vec in X:
        if len(vec) != ref_len:
            raise KeyError("No todos los vectores tienen la misma dimension")
    #TODO Paso 1: Crear una matriz de 0s con las dimensiones de la matriz de distancias. nxn donde n es el num. de vectores.
    matrix = [[0. for _ in range(len(X))] for _ in range(len(X))]
    #TODO: Llenar la matriz usando la función que acabamos de crear.
    for i in range(len(X)):
        for j in range(len(X)):
            matrix[i][j] = eucledian_distance_nans(X[i],X[j])
    return matrix
    

In [38]:
vec_4 = [0., None, 10., 17., 20., 32., 5., 0.]
vec_5 = [15., 120., 10., None, 3., 1., 0., 0.]
matrix_1 = [vec_1, vec_2, vec_3, vec_4, vec_5]

print("Matriz:")
for i in matrix_1:
    print(i)
print("\n")

dist_matrix = calculate_distance_matrix(matrix_1)
print("Matriz de distancias:")
for i in dist_matrix:
    print(i)

# LA RESPUESTA DEBE SER:
# Matriz de distancias:
# [0.0, 9.46572765, 129.0736224, 44.01817806, 128.47308408]
# [9.46572765, 0.0, 24.11875382, 42.8285619, 23.52303835]
# [129.0736224, 24.11875382, 0.0, 42.68154502, 19.30210943]
# [44.01817806, 42.8285619, 42.68154502, 0.0, 44.72135955]
# [128.47308408, 23.52303835, 19.30210943, 44.72135955, 0.0]

Matriz:
[0.0, 10.0, 5.0, None, 5.0, 1.0, 5.0, None]
[1.0, None, 0.0, 7.0, 0.0, 2.0, 3.0, 10.0]
[15.0, 120.0, 10.0, 1.0, 2.0, 7.0, 15.0, 8.0]
[0.0, None, 10.0, 17.0, 20.0, 32.0, 5.0, 0.0]
[15.0, 120.0, 10.0, None, 3.0, 1.0, 0.0, 0.0]


Matriz de distancias:
[0.0, 9.46572765, 129.0736224, 44.01817806, 128.47308408]
[9.46572765, 0.0, 24.11875382, 42.8285619, 23.52303835]
[129.0736224, 24.11875382, 0.0, 42.68154502, 19.30210943]
[44.01817806, 42.8285619, 42.68154502, 0.0, 44.72135955]
[128.47308408, 23.52303835, 19.30210943, 44.72135955, 0.0]


#### 2. Cálculo de los k vecinos más cercanos

In [39]:
def generar_matriz_vecinos(k, X, X_dist):
    """
    Genera una matriz que guarda el indice de los k vecinos más cercanos de cada vector en la matriz de distancias.
    """
    # TODO Paso 1: Crear una matriz de 0s con las dimensiones de la matriz de vecinos. nxk donde n es el num. de vectores.
    matriz_vecinos = [[0 for _ in range(k)] for _ in range(len(X))]
    
    # Paso 2: Llenar la matriz usando la matriz de distancias.
    for i in range(len(X)):
        # TODO :Obtener los indices de los k vecinos más cercanos a X[i] ordenados por distancia. Recuerde eliminar el indice del propio vector !
        indices_vecinos = sorted(list(range(len(X_dist[i]))), key=lambda x: X_dist[i][x])
        indices_vecinos.remove(i)
        indices_vecinos = indices_vecinos[:k]

        matriz_vecinos[i] = indices_vecinos
    
    return matriz_vecinos
 

In [40]:
print(generar_matriz_vecinos(2, matrix_1, dist_matrix))
# LA RESPUESTA DEBE SER:
# [[1, 3], [0, 4], [4, 1], [2, 1], [2, 1]]

[[1, 3], [0, 4], [4, 1], [2, 1], [2, 1]]


#### 3. Imputar los valores faltantes usando la media de los k vecinos más cercanos

In [41]:
def knn_eucledian_imputer(X, k):
    """
    Imputa los valores faltantes en la matriz X utilizando el algoritmo KNN.
    """
    # Paso 1: Calcular la matriz de distancias entre los vectores de X.
    dist_matrix = calculate_distance_matrix(X)
    # Paso 2: Generar la matriz de indices de los k vecinos más cercanos.
    vecinos = generar_matriz_vecinos(k, X, dist_matrix)
    # Paso 3: Crear una copia de X para almacenar los valores imputados.
    X_imputed = copy.deepcopy(X)

    # Paso 4: Iterar sobre cada vector en X.
    for idx in range(len(X)):
        # Paso 5: Iterar sobre cada elemento del vector.
        for jdx in range(len(X[idx])):
            # Si el elemento es NaN, imputar su valor.
            if X[idx][jdx] is None:
                # TODO Paso 6: Obtener los valores de los k vecinos más cercanos.
                indices_vecinos = vecinos[idx]
                valores_vecinos_nonan = [X[vecino][jdx] for vecino in indices_vecinos if X[vecino][jdx] is not None]
                # TODO Paso 7: Calcular la media de los valores de los vecinos y asignarla al elemento faltante. Ojo, considerar la excepción de que no haya vecinos con valores no NaN.
                if valores_vecinos_nonan:
                    X_imputed[idx][jdx] = sum(valores_vecinos_nonan) / len(valores_vecinos_nonan)
                else:
                    X_imputed[idx][jdx] = None
    
    return X_imputed

In [42]:
imputed_matrix = knn_eucledian_imputer(matrix_1, 2)
print("Matriz imputada:")
for i in imputed_matrix:
    print(i)

# LA RESPUESTA DEBE SER:
# [0.0, 10.0, 5.0, 12.0, 5.0, 1.0, 5.0, 5.0]
# [1.0, 65.0, 0.0, 7.0, 0.0, 2.0, 3.0, 10.0]
# [15.0, 120.0, 10.0, 1.0, 2.0, 7.0, 15.0, 8.0]
# [0.0, 92.5, 10.0, 17.0, 20.0, 32.0, 5.0, 0.0]
# [15.0, 120.0, 10.0, 4.0, 3.0, 1.0, 0.0, 0.0]

Matriz imputada:
[0.0, 10.0, 5.0, 12.0, 5.0, 1.0, 5.0, 5.0]
[1.0, 65.0, 0.0, 7.0, 0.0, 2.0, 3.0, 10.0]
[15.0, 120.0, 10.0, 1.0, 2.0, 7.0, 15.0, 8.0]
[0.0, 120.0, 10.0, 17.0, 20.0, 32.0, 5.0, 0.0]
[15.0, 120.0, 10.0, 4.0, 3.0, 1.0, 0.0, 0.0]


#### **Probemos nuestra implementación en los datos reales!**

In [43]:
marengo_lists = marengo_df.iloc[:,4:].values.tolist()
marengo_lists = [[None if pd.isna(x) else x for x in row] for row in marengo_lists]
start_time = time.time()
imputed_marengo = knn_eucledian_imputer(marengo_lists, 10)
end_time = time.time()
print(f"Tiempo de ejecución: {end_time - start_time:.2f} segundos")
print("Matriz imputada de Marengo:")
for i in imputed_marengo[:10]:  # Imprimimos solo las primeras 10 filas por brevedad
    print(i)

Tiempo de ejecución: 8.30 segundos
Matriz imputada de Marengo:
[13.508667, 19.367, 8.467, 23.900002, 42680.883, 13.708391, 35.28, 20.6, 3.4468565]
[14.596166, 19.317001, 11.167, 1.5699999539999998, 42684.59, 13.493999, 33.48, 19.57375, 3.663233]
[13.248249, 20.417, 8.217, 1.7, 42688.617, 10.446206, 30.599998, 23.35, 4.0588975]
[12.804502, 19.417, 7.467, 1.3000001, 42692.95, 14.05845, 37.44, 25.6, 4.251072]
[12.331582, 18.767, 7.174142714285714, 0.0, 42697.59, 15.39184, 36.36, 23.75, 3.9684093]
[12.266999, 19.94557157142857, 6.87325, 0.0, 42702.523, 15.119999, 38.519997, 24.43, 3.959472]
[12.875333, 21.117, 5.817, 0.1, 42707.75, 14.320731649999999, 23.039999, 25.17, 4.401479]
[11.935749, 20.267, 4.467, 0.0, 42713.258, 15.876775, 40.8599985, 26.24, 4.566951]
[13.164916, 20.667, 7.1670003, 0.0, 42719.04, 19.164717, 46.44, 23.83, 4.1956615]
[12.521167, 18.667, 6.517, 0.0, 42725.094, 23.277834, 53.639996, 26.48, 4.431484]


### Ex.2: Contruir un pipeline para aplicar KNN-I usando scikit-learn.

In [ ]:
# TODO Escalar los datos para mejorar la precisión del KNN usando la clase StandardScaler de sklearn.
scaler = StandardScaler()
scaled_data = scaler.fit_transform(marengo_df.iloc[:, 4:])
# TODO usar la implementacion del algoritmo KNN de sklearn para imputar los valores faltantes de la matriz de Marengo.
# Este es el ejercicio más sencillo, ya que sklearn se encarga de todo el proceso de cálculo de distancias y vecinos.
# Puede resolverse usando solo dos lineas de codigo, una para crear una instacia de la clase KNNImputer y otra para imputar los valores faltantes.
imputer = KNNImputer(n_neighbors=2)
imputed_array = imputer.fit_transform(marengo_df.iloc[:, 4:])
# Invertir el escalado para que los datos imputados estén en la misma escala que los originales.
imputed_array = scaler.inverse_transform(imputed_array)
# Tomar los resultados y convertirlos de nuevo en DataFrame :)
imputed_df = pd.DataFrame(imputed_array, columns=marengo_df.iloc[:, 4:].columns)
imputed_df.head(10)

,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration
0,24.960969,47.155395,21.233263,126.450245,2.876964e+07,75.582353,350.349306,87.409488,5.692513
1,25.890462,47.083797,25.291172,6.722824,2.877213e+07,74.619409,334.297042,89.156505,5.824552
2,24.738388,48.658992,20.857531,12.129868,2.877484e+07,60.930213,308.613401,96.389483,6.065998
3,24.359115,47.226995,19.730334,10.070042,2.877776e+07,77.154647,369.612023,103.736751,6.183268
4,23.954907,46.296197,16.574183,3.375605,2.878088e+07,83.143583,359.980665,97.695664,6.010779
5,23.899707,50.234188,17.701380,3.375605,2.878420e+07,81.922606,379.243355,99.916171,6.005325
6,24.419655,49.661390,17.250501,3.890562,2.878772e+07,97.275393,241.193899,102.332606,6.275051
7,23.616586,48.444192,15.221546,3.375605,2.879143e+07,85.321674,395.295628,105.826641,6.376026
8,24.667163,49.016991,19.279456,3.375605,2.879532e+07,100.089502,449.873345,97.956900,6.149455
9,24.116947,46.152998,18.302551,3.375605,2.879939e+07,118.563613,514.082367,106.610349,6.293361


### Ex.3: Implementar MissForest usando los bloques de construcción que ofrece scikit-learn.

In [50]:
# TODO Escalar los datos
scaler = StandardScaler()
scaled_data = scaler.fit_transform(marengo_df.iloc[:, 4:])

# TODO Imputar usando IterativeImputer con RandomForestRegressor
imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=10, random_state=42), 
                           max_iter=100, random_state=42)
imputed_scaled = imputer.fit_transform(scaled_data)

imputed_data = scaler.inverse_transform(imputed_scaled)
imputed_df = pd.DataFrame(imputed_data, columns=marengo_df.iloc[:, 4:].columns)

imputed_df.head(10)

/home/julagu/miniconda3/envs/hydro_imputation/lib/python3.10/site-packages/sklearn/impute/_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration
0,13.508667,19.367000,8.467,23.900002,42680.883,13.708391,35.280000,20.600,3.446856
1,14.596166,19.317001,11.167,2.230000,42684.590,13.493999,33.480000,20.553,3.663233
2,13.248249,20.417000,8.217,1.700000,42688.617,10.446206,30.599998,23.350,4.058897
3,12.804502,19.417000,7.467,1.300000,42692.950,14.058450,37.440000,25.600,4.251072
4,12.331582,18.767000,6.127,0.000000,42697.590,15.391840,36.360000,23.750,3.968409
5,12.266999,18.597001,5.957,0.000000,42702.523,15.119999,38.519997,24.430,3.959472
6,12.875333,21.117000,5.817,0.100000,42707.750,8.422382,23.039999,25.170,4.401479
7,11.935749,20.267000,4.467,0.000000,42713.258,15.876775,39.023999,26.240,4.566951
8,13.164916,20.667000,7.167,0.000000,42719.040,19.164717,46.440000,23.830,4.195661
9,12.521167,18.667000,6.517,0.000000,42725.094,23.277834,53.639996,26.480,4.431484


### Ex4. Elegir un conjunto de metricas y comparar las 2 tecnicas !

In [ ]:
# TODO Ver la documentacion de Sklearn y utilizar metricas de regresión para evaluar la calidad de la imputación. Recuerden que tenemos los datos reales !!!